In [ ]:
%load_ext autoreload
%autoreload 2

import os
import matplotlib.pyplot as plt

# Change working directory to project root
os.chdir('../../')

import torch
import numpy as np
from pathlib import Path
from tqdm import tqdm
import json
import imageio
from hmr4d.utils.wis3d_utils import make_wis3d
from hmr4d.utils.smplx_utils import make_smplx

from hmr4d.utils.geo_transform import apply_T_on_points
from pytorch3d.transforms import axis_angle_to_matrix, matrix_to_axis_angle, matrix_to_rotation_6d

In [ ]:
from hmr4d.dataset.rich.rich_smpl_test import RichSmplFullSeqDataset
dataset = RichSmplFullSeqDataset(vid_presets=None)
wis3d = make_wis3d(name='debug-noise-aug')

In [ ]:
data = dataset[0]
gender = data["gender"]

smplx_model = dataset.rich_smplx[gender]
faces = smplx_model.bm.faces

In [ ]:
# GT
smplx_out = smplx_model(body_pose=data["gt_smpl_params"]["body_pose"])
seq_length = len(smplx_out.vertices)
for i in range(seq_length):
    wis3d.set_scene_id(i)
    wis3d.add_mesh(vertices=smplx_out.vertices[i], faces=faces, name="gt-smplx")

In [ ]:
# CLIFF
smplx_out = smplx_model(body_pose=data["obs_smpl_params"]["body_pose"])
seq_length = len(smplx_out.vertices)
for i in range(seq_length):
    wis3d.set_scene_id(i)
    wis3d.add_mesh(vertices=smplx_out.vertices[i], faces=faces, name="obs-smplx")

In [ ]:
gt = data["gt_smpl_params"]["body_pose"].reshape(-1, 21, 3)
obs = data["obs_smpl_params"]["body_pose"].reshape(-1, 21, 3)

gt_rotmat = axis_angle_to_matrix(gt)  # (L, J, 3)
obs_rotmat = axis_angle_to_matrix(obs)  # (L, J, 3)
# diff_rotmat = obs_rotmat @ gt_rotmat.transpose(-1, -2)
diff_rotmat = gt_rotmat.transpose(-1, -2) @ obs_rotmat

diff_aa = matrix_to_axis_angle(diff_rotmat)  # (L, J, 3)
diff_unit_axis = diff_aa / (torch.norm(diff_aa, dim=-1, keepdim=True) + 1e-6)
diff_unit_angle = torch.norm(diff_aa, dim=-1, keepdim=True)

In [ ]:
from hmr4d.utils.geo.augment_noisy_pose import gaussian_augment

std_angle = 10
std_angle = [10] * 15 + [20] * 6
noisy_gt = gaussian_augment(gt, std_angle=std_angle, to_R=True)[2]

# GT + noisy
smplx_out = smplx_model(body_pose=noisy_gt)
seq_length = len(smplx_out.vertices)
for i in range(seq_length):
    wis3d.set_scene_id(i)
    wis3d.add_mesh(vertices=smplx_out.vertices[i], faces=faces, name="gt+noise-smplx")

### Draw Statistic

In [ ]:
unit_axis = []
unit_angle = []

for idx in tqdm(range(len(dataset))):
    data = dataset[idx]

    gt = data["gt_smpl_params"]["body_pose"].reshape(-1, 21, 3)
    obs = data["obs_smpl_params"]["body_pose"].reshape(-1, 21, 3)

    gt_rotmat = axis_angle_to_matrix(gt)  # (L, J, 3)
    obs_rotmat = axis_angle_to_matrix(obs)  # (L, J, 3)
    # diff_rotmat = obs_rotmat @ gt_rotmat.transpose(-1, -2)
    diff_rotmat = gt_rotmat.transpose(-1, -2) @ obs_rotmat

    diff_aa = matrix_to_axis_angle(diff_rotmat)  # (L, J, 3)
    diff_unit_axis = diff_aa / (torch.norm(diff_aa, dim=-1, keepdim=True) + 1e-6)
    diff_unit_angle = torch.norm(diff_aa, dim=-1, keepdim=True)

    unit_axis.append(diff_unit_axis)
    unit_angle.append(diff_unit_angle)

unit_axis = torch.cat(unit_axis, dim=0)
unit_angle = torch.cat(unit_angle, dim=0)

In [ ]:
plt.rc("font", size=8)
plt.figure(figsize=(10, 5))  # Adjust the size of the figure as needed
for j in range(21):
    plt.subplot(3, 7, j + 1)  # Create a subplot in a 5x5 grid
    plt.hist(diff_unit_angle[:, j].numpy(), bins=30, alpha=0.75)
    plt.xlim([0, np.pi / 2])
    plt.ylim([0, 150])
    plt.title(f"J-{j+1}", fontsize=7)
plt.tight_layout()  # Adjust the layout so that the subplots do not overlap
plt.show()

In [ ]:
plt.rc("font", size=8)
fig = plt.figure(figsize=(10, 5))  # Adjust the size of the figure as needed
for j in range(21):
    ax = fig.add_subplot(3, 7, j + 1, projection='3d')  # Create a subplot in a 3x7 grid
    ax.scatter(diff_unit_axis[:, j, 0].numpy(), diff_unit_axis[:, j, 1].numpy(), diff_unit_axis[:, j, 2].numpy())
    ax.set_title(f"J-{j+1}", fontsize=8)
plt.tight_layout()  # Adjust the layout so that the subplots do not overlap
plt.show()